In [ ]:
def preprocessing(target):
    target = re.sub(r'\n(?!\|)', ', ', target)
    return target.replace('*','').replace('[', '').replace(']', '')

def getAttribute(attribute, target):
    target = preprocessing(target)
    matchingString = f'\|{attribute}=([^\n\{{\}}<>]*)[^\n]*\n\|'
    result="?"
    if attribute == "Mort" or attribute == "Naissance":
        try:
            result = re.search('\|{}=.*?(\d{4}).*?\s\|',target).group(1)
        except:
            return "0"
    else:
        #if attribute == "Sang":
            #matchingString = "\|Sang=\[\[([^]]+)\]\]"
        #if attribute == "Localisation":
            #matchingString = "\|Localisation=.*?\[\[([^]]+)\]\]"
        #if attribute == "Statut":
            #matchingString = "\|Statut=([^\{\}\n]*).*\n\|"
        try:
            result = re.search(matchingString, target).group(1)
            if result == "":
                result="?"
        except:
            return "?"
    result = result.replace("'","('')")
    return result

def getNom(target):
    try:
        result = re.search('\|Nom=(.*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getYeux(target):
    try:
        result = re.search('\|Yeux=([\w\ 0-9,\']*).*?\s\|', target).group(1)
    except:
        return ""
    return result

def getGenre(target):
    try:
        result = re.search('\|Genre=\ ?([A-Za-z0-9â]*?)\s\|', target).group(1)
    except:
        return ""
    return result

def getCheveux(target):
    try:
        result = re.search('\|Cheveux=([A-Za-z0-9]*?)\s\|', target).group(1)
        if result == "":
            return "Inconnu"
    except:
        return "Inconnu"
    return result


def getNaissance(target):
    try:
        result = re.search('\|Naissance=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result

def getMort(target):
    try:
        result = re.search('\|Mort=.*?(\d{4}).*?\s\|',target).group(1)
    except:
        return "0"
    return result


# with open('personnagesnaissance.txt','x') as f:
#     for page in pages:
#         if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
#             f.write(getAttribute("Nom",page.text) +" "+ getAttribute("Naissance",page.text)+"\n")
for page in pages[0:100]:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        id = ""
        for attr in ["Nom","Cheveux","Yeux","Genre","Patronus","Sang"]:
            id += f" {getAttribute(attr,page.text)}"
        id += str(getMort(page.text))
        print(id)
for page in pages[100:2000]:
    if "[[Catégorie:Créatures" in page.text and ("Catégorie:" not in page.title.text and "Portail:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        yeux = getAttribute("Yeux",page.text)
        statut = getAttribute("Statut",page.text)
        apparence = getAttribute("Apparence",page.text)
        caractère = getAttribute("Caractère",page.text)
        localisation = getAttribute("Localisation",page.text)
        print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")

In [ ]:
with open('personnages.txt','x') as f:
    for page in pages:
        if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
            f.write(xmlToSql.getAttribute("Nom",page.text) +" "+ xmlToSql.getAttribute("Yeux",page.text)+"\n")

In [ ]:
from bs4 import BeautifulSoup
import re
from xmlToSql import getAttribute
import sqlite3

Reading and parsing the dump with BeautifulSoup

In [ ]:
with open('frharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')

In [ ]:
pages = soup.find_all('page')

## Creating the databases

Start by creating an empty database.

In [ ]:
connection = sqlite3.connect("harrypotter_fr.db")
cursor = connection.cursor()

### First the character table

Create the table for characters named "personnages".

Fill in the records thanks to the dump of the french Harry Potter Fandom and parsing functions in the xmlToSql.py file.

In [ ]:
cursor.execute("CREATE TABLE personnages (nom TEXT NOT NULL PRIMARY KEY, genre TEXT, yeux TEXT, cheveux TEXT, naissance INTEGER, mort INTEGER, patronus TEXT, sang TEXT)")
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        yeux = getAttribute("Yeux",page.text)
        cheveux = getAttribute("Cheveux",page.text)
        genre = getAttribute("Genre",page.text)
        patronus = getAttribute("Patronus",page.text)
        naissance = getAttribute("Naissance",page.text)
        mort = getAttribute("Mort",page.text)
        sang = getAttribute("Sang",page.text)
        print(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            cursor.execute(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

### Then table for creatures

In [ ]:
cursor.execute("CREATE TABLE créatures (nom TEXT NOT NULL PRIMARY KEY, statut TEXT, yeux TEXT, apparence TEXT, caractère TEXT, localisation TEXT)")

In [ ]:
for page in pages:
    if "[[Catégorie:Créatures" in page.text and ("Catégorie:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        yeux = getAttribute("Yeux",page.text)
        statut = getAttribute("Statut",page.text)
        apparence = getAttribute("Apparence",page.text)
        caractère = getAttribute("Caractère",page.text)
        localisation = getAttribute("Localisation",page.text)
        print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")
        try:
            cursor.execute(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")
        except:
            print(f"Already inserted {nom}")

## Finally Junction Table between family members

In [ ]:
cursor.execute("CREATE TABLE famille (premier TEXT NOT NULL, second TEXT NOT NULL, relation TEXT, FOREIGN KEY (premier) REFERENCES personnages(nom), FOREIGN KEY (second) REFERENCES personnages(nom))")

In [ ]:
for page in pages[0:100]:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text and "Utilisateur:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        print(nom + " is related to: ")
        familleDiv = re.search(f'\|Famille=(.*?)\|Sang',page.text,re.DOTALL).group(1)
        famille = re.findall(r'\*\[\[([^\]]+)\]\]',familleDiv)
        print(famille)
        try:
            cursor.execute(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

In [ ]:
def insertRelative(first,second,relation):
    first = first.replace("'","('')")
    second = second.replace("'","('')")
    relation = relation.replace("'","('')")
    try:
        cursor.execute(f"Insert INTO famille VALUES('{first}','{second}','{relation}')")
    except:
        print(f"Inserting {first} {second} {relation} didn't work")
    return

pattern = re.compile(r'Famille\s*=\s*(.*?)\s\|', re.S)
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text and "Utilisateur:" not in page.title.text):
        nom = getAttribute("Nom",page.text)
        
        if re.search('Famille\s*=\s*\n\|',page.text): #Related to no-one
            continue
        singleRelative = re.search('Famille\s*=\s*\[\[(.*?)\]\]',page.text)
        if singleRelative:#Related to one person
            onlyRelative = singleRelative.group(1)
            if "|" in onlyRelative:
                parts = onlyRelative.split("|")
                insertRelative(parts[0],nom,parts[1])
            else:
                relation = re.search('Famille=.*?{{Comment\|(.*?)}}',page.text)
                if relation:
                    insertRelative(onlyRelative,nom,relation.group(1))
                else:
                    insertRelative(onlyRelative,nom,"?")
            continue
        multipleRelatives = pattern.search(page.text)
        if multipleRelatives:
            famille_text = multipleRelatives.group(1)
            famille = re.findall(r'\*\[\[([^\]]+)\]\]',famille_text)
            for member in famille:
                if "|" in member:
                    parts = member.split("|")
                    insertRelative(parts[0],nom,parts[1])
                else:
                    relation = re.search(f'\*\[\[{member}\]\].*?{{{{Comment\|(.*?)}}}}',page.text)
                    if relation:
                        insertRelative(member,nom, relation.group(1))
            
        

In [ ]:
try:
    cursor.execute(f"Insert INTO famille VALUES('Ronul Weasley','Hffar','best friend')")
except:
    print(f"Already inserted {nom}")

In [ ]:
cursor.execute("SELECT * FROM famille WHERE second='Harry Potter'")

# Fetch all rows from the query result
entries = cursor.fetchall()

# Iterate through the rows and print each entry
for entry in entries:
    print(entry)



Finally commit the changes to the database so that all changes are saved and close the connection

In [ ]:
connection.commit()
cursor.close()
connection.close()